In [1]:
import email, codecs
import os,pg8000
import numpy as np
import pandas as pd

In [3]:
db_config=open('./db_config')
dbHOST=db_config.readline().split('\"')[1]
dbUSER=db_config.readline().split('\"')[1]
dbPASS=db_config.readline().split('\"')[1]
dbDB=db_config.readline().split('\"')[1]
db_config.close()
conn = pg8000.connect(host=dbHOST,user=dbUSER,password=dbPASS,database=dbDB)

def escape(s):
    return asciiCodify(s).replace("'", "''").replace('%', '%%')


In [ ]:
def insertContent(path,PJstatus,messages,conn,need_body):
#try:


    #project='ApacheMailArchives'
    listname=PJstatus[1]
    cursor = conn.cursor()


    listidIn=PJstatus[0] 

    # get all of the aliases and messageids currently in the database
    insertedMessageIds = set()
    #sql = "select messageid from messages"+TABLE_POSTFIX+" where list = '"+str(listidIn)+"'"
    try:
        cursor.execute(_("select message_id from messages"+TABLE_POSTFIX+" where thread_id = '"+str(listidIn)+"'"))
        result=cursor.fetchall()
    except pg8000.ProgrammingError,e:
        print e
        
    for row in result:
    insertedMessageIds.add(row[0])

    aliases = {}
    cursor.execute("select name, email, aliasid from aliases"+TABLE_POSTFIX)
    result=cursor.fetchall()
    for name, address, aliasId in result:
    aliases[name, address] = aliasId


    #############################################################first insert all of the aliases
    unknownAliases = set()
    for message in messages:
    if "from" in message:
        name, address = email.utils.parseaddr(message["from"])
        name = asciiCodify(name)
        if not aliases.has_key( (name, address) ):
             unknownAliases.add( (name, address) )

    if len(unknownAliases) > 0:
    sql = "insert into aliases"+TABLE_POSTFIX+" (aliasid, name, email) values " + \
        ", ".join(["(DEFAULT, '" + escape(name) + "', '" + escape(address) + "')" for name, address in unknownAliases]) + \
        " returning name, email, aliasid"
    #               print sql
    cursor.execute(sql)
    result=cursor.fetchall()
    for name, address, id in result:
        aliases[name, address] = id
    conn.commit()

    #          print aliases

    errorsLog = open("errors", "w")

    ########################################################### Now insert all of the messages
    for message in messages:
    if not "message-id" in message:
        continue
    messageId = fixMsgId(message["message-id"])
    #               print messageId
    if not "from" in message:
        print messageId, "no from in message"
        print >> errorsLog, message, "no from in message"
        continue

    # if we've already put it in the database then skip it
    if messageId in insertedMessageIds:
        continue
    insertedMessageIds.add(messageId)
    name, address = email.utils.parseaddr(message["from"])
    name = asciiCodify(name)
    aliasId = aliases[name, address]

    if message.is_multipart():
        body = "EMPTY"
        for subMessage in message.get_payload():
             if subMessage.get_content_type() == "text/plain":
                  body = escape(subMessage.get_payload())
                  break;
    else:
        body = escape(message.get_payload())

    subject = escape(message["subject"])
    try:
        if len(message["date"].split(" "))>4:
             datetime = message["date"]
        if '(' in datetime:
             delLen=-len(datetime.split("(")[1])-2
             datetime=datetime[:delLen]
        tempdate=str(re.search(r'([0-9]+) ([\w]+) ([0-9]+) ([0-9]+):([0-9]+):([0-9]+) ([\W]+[0-9]+)', str(datetime)).group())
        tempdate=tempdate.split(' ')
        if len(tempdate[0])<4 and len(tempdate[2])<4:
             tempdate[2]="20"+tempdate[2]
             datetime=' '.join(tempdate)
    except:
        #print message
        # Date not in usual placement, try this placement
        # Extracting date of type "From nobody Sat Mar 17 14:02:11 2012"
        datetime = str(re.search(r'([0-9]+) ([\w]+) ([0-9]+) ([0-9]+):([0-9]+):([0-9]+) ([\W]+[0-9]+)', str(message)).group())



    inReplyTo = "NULL"
    if "in-reply-to" in message:
        inReplyTo = escape(fixMsgId(message["in-reply-to"]))
    if need_body==True:
       sql = "insert into messages"+TABLE_POSTFIX+" (list, messageid, subject, body, senderaliasid, datetime, referenceid,senderalias) values "+\
        "('"+str(listidIn)+"', '"+str(messageId)+"', '"+str(subject)+"', '"+str(body)+"', '"+str(aliasId)+"','"+datetime+"', '"+inReplyTo+"','"+str(name)+"')"
    else:
       sql = "insert into messages"+TABLE_POSTFIX+" (list, messageid, subject, senderaliasid, datetime, referenceid,senderalias) values "+\
        "('"+str(listidIn)+"', '"+str(messageId)+"', '"+str(subject)+"', '"+str(aliasId)+"','"+datetime+"', '"+inReplyTo+"','"+str(name)+"')"

    #               print "Inserting", messageId

    try:
        cursor.execute(sql)

    except pg8000.ProgrammingError,e:
        #print message
        print "insert error"
        print >> errorsLog,e
        print >> errorsLog, sql
    conn.commit()